In [1]:
import pandas as pd
from sqlalchemy import create_engine

# 데이터베이스 연결 설정
db_host = '192.168.0.41:5432'
db_user = 'postgres'
db_password = '1234'
db_name = 'postgres'

# 데이터베이스 연결
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}/{db_name}')

# 데이터베이스에서 데이터 가져오기
query = f"SELECT * FROM finance_data order by name, date"
df_or = pd.read_sql(query, engine)

In [2]:
# 선정된 종목들만 선택
from tqdm import tqdm
df_list = []
for name in tqdm(df_or['name'].unique()):
    df_list.append(df_or[df_or['name']==name])

100%|██████████| 2593/2593 [05:13<00:00,  8.27it/s]


In [3]:
import warnings
warnings.filterwarnings("ignore")
# 이동평균선(5일, 20일, 60일) 추가
for df in df_list:
    df['5MA'] = df['close'].rolling(window=5).mean()
    df['20MA'] = df['close'].rolling(window=20).mean()
    df['60MA'] = df['close'].rolling(window=60).mean()

In [5]:
df_list[0]

,date,code,name,open,high,low,close,volume,marketcap,5MA,20MA,60MA
0,2019-01-04,060310,3S,2180,2395,2130,2240,180908,9.944677e+10,NaN,NaN,NaN
1,2019-01-07,060310,3S,2260,2315,2225,2260,77610,1.003347e+11,NaN,NaN,NaN
2,2019-01-08,060310,3S,2230,2265,2200,2250,39095,9.989073e+10,NaN,NaN,NaN
3,2019-01-09,060310,3S,2260,2260,2210,2230,35077,9.900281e+10,NaN,NaN,NaN
4,2019-01-10,060310,3S,2205,2245,2205,2230,20763,9.900281e+10,2242.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1097,2023-06-15,060310,3S,3055,3150,2970,3025,3540015,NaN,3021.0,2726.75,2404.166667
1098,2023-06-16,060310,3S,3035,3060,2960,2960,1493930,NaN,3020.0,2763.00,2418.666667
1099,2023-06-19,060310,3S,2990,3125,2970,3070,3473647,NaN,3019.0,2802.75,2434.916667
1100,2023-06-20,060310,3S,3055,3140,2990,3005,2309234,NaN,3012.0,2838.50,2450.166667


In [127]:
# ## XGBoost Regressor
# import numpy as np
# import pandas as pd
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from xgboost import plot_importance
# import matplotlib.pyplot as plt

# # 데이터 불러오기
# data = df_list[0]
# # 종속 변수 설정
# target = 'close'
# # 독립 변수 선택
# features = ['open', 'high', 'low', 'close', 'volume', 'marketcap', '5MA', '20MA', '60MA']
# # 데이터 분리 (Train/Test)
# train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)
# # 데이터 전처리
# scaler = MinMaxScaler()
# train_scaled = scaler.fit_transform(train_data[features])[5:-5]
# test_scaled = scaler.transform(test_data[features])[5:-5]
# print(train_scaled, test_scaled)
# # 타겟 변수 설정 (이틀 뒤의 close 값)
# y_train = train_data[target].shift(-5).dropna()[:-5]  # 마지막 두 날짜 제외
# y_test = test_data[target].shift(-5).dropna()[:-5]  # 마지막 두 날짜 제외
# print(y_train,y_test)


# # XGBoost 모델 파라미터 설정
# params = {
#     'objective': 'reg:squarederror',  # 회귀 문제 설정
#     'max_depth': 5,  # 트리의 최대 깊이
#     'learning_rate': 0.1,  # 학습률
#     'n_estimators': 100  # 트리의 개수
# }

# model = xgb.XGBRegressor().fit(train_scaled, y_train)



# # 테스트 데이터에 대한 예측 수행
# y_pred = model.predict(test_scaled)

In [128]:
# import numpy as np
# import pandas as pd
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from xgboost import plot_importance
# import matplotlib.pyplot as plt

# # 데이터 불러오기
# data = df_list[1]
# # 종속 변수 설정
# target = 'target'
# # 독립 변수 선택
# features = ['open', 'high', 'low', 'close', 'volume', 'marketcap', '5MA', '20MA', '60MA']
# # 데이터 분리 (Train/Test)
# data['target'] = data['close'].shift(-5)
# data.dropna(inplace=True)
# train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)
# # 데이터 전처리
# scaler = MinMaxScaler()
# train_scaled = scaler.fit_transform(train_data[features])
# test_scaled = scaler.transform(test_data[features])
# print(train_scaled, test_scaled)
# # 타겟 변수 설정 (이틀 뒤의 close 값)
# y_train = train_data[target]
# y_test = test_data[target]
# print(y_train,y_test)

# # 데이터셋과 레이블 데이터의 크기 확인
# print("Train dataset size:", len(train_scaled))
# print("Train label size:", len(y_train))
# print("Test dataset size:", len(test_scaled))
# print("Test label size:", len(y_test))

# # XGBoost 모델 파라미터 설정
# params = {
#     'objective': 'reg:squarederror',  # 회귀 문제 설정
#     'max_depth': 5,  # 트리의 최대 깊이
#     'learning_rate': 0.1,  # 학습률
#     'n_estimators': 100  # 트리의 개수
# }

# model = xgb.XGBRegressor().fit(train_scaled, y_train)



# # 테스트 데이터에 대한 예측 수행
# y_pred = model.predict(test_scaled)



In [129]:
# import matplotlib.pyplot as plt
# import numpy as np

# importance = model.get_booster().get_score(importance_type='weight')  # 피처의 중요도 가져오기

# # 피처와 중요도를 매핑하는 딕셔너리 생성
# feature_names = features
# feature_importance = {feature_names[int(k[1:])]: v for k, v in importance.items()}

# # 피처 중요도를 기준으로 내림차순으로 정렬
# sorted_importance = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)

# # 시각화
# # plt.figure(figsize=(10, 6))
# # plt.barh(range(len(sorted_importance)), [x[1] for x in sorted_importance], align='center')
# # plt.yticks(range(len(sorted_importance)), [x[0] for x in sorted_importance])
# # plt.xlabel('Feature Importance')
# # plt.ylabel('Features')
# # plt.title('XGBoost Feature Importance')
# # plt.show()


In [124]:
## LSTM 이중분류
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 데이터 불러오기
data = df_list[0]
# 종속 변수 설정
target = 'target'
# 독립 변수 선택
features = ['open', 'high', 'low', 'close', 'volume', 'marketcap', '5MA', '20MA', '60MA']

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[features])

# 타겟 변수 생성 (이틀 뒤의 close 값 비교)
data[target] = (data['close'].shift(-5) > data['close']).astype(int)

# 데이터 분리 (Train/Test)
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

# 독립 변수 데이터
train_scaled = scaled_data[:len(train_data)]
test_scaled = scaled_data[len(train_data):]

# 타겟 변수 데이터
y_train = train_data[target].values[5:-5]
y_test = test_data[target].values[5:-5]

# 데이터셋과 레이블 데이터의 크기 확인
print("Train dataset size:", len(train_scaled))
print("Train label size:", len(y_train))
print("Test dataset size:", len(test_scaled))
print("Test label size:", len(y_test))

# LSTM 모델 구성
model = Sequential()
model.add(LSTM(64, input_shape=(train_scaled.shape[1], 1)))  # LSTM 층, 입력 크기는 (타임스텝 수, 피처 수)
model.add(Dense(1, activation='sigmoid'))  # 이진 분류를 위한 밀집 층

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
model.fit(train_scaled.reshape((train_scaled.shape[0], train_scaled.shape[1], 1)), y_train, epochs=100, batch_size=1, verbose=0)

# 예측 수행
y_pred = model.predict(test_scaled.reshape((test_scaled.shape[0], test_scaled.shape[1], 1)))

# 예측 결과 출력
print(y_pred)


Train dataset size: 774
Train label size: 774
Test dataset size: 186
Test label size: 186
6/6 [==============================] - 0s 2ms/step
[[0.52956414]
 [0.52126765]
 [0.563197  ]
 [0.5460012 ]
 [0.5487512 ]
 [0.5238357 ]
 [0.49437526]
 [0.49063   ]
 [0.46471295]
 [0.471155  ]
 [0.47943002]
 [0.46546215]
 [0.47433785]
 [0.46988702]
 [0.45813683]
 [0.4668025 ]
 [0.4703884 ]
 [0.46314406]
 [0.46768707]
 [0.47032216]
 [0.4638523 ]
 [0.4729523 ]
 [0.510127  ]
 [0.5393983 ]
 [0.51232135]
 [0.50581694]
 [0.5038049 ]
 [0.50688297]
 [0.49921453]
 [0.502303  ]
 [0.5503038 ]
 [0.6163868 ]
 [0.63631874]
 [0.63434535]
 [0.5963257 ]
 [0.7781808 ]
 [0.6153911 ]
 [0.63550484]
 [0.5916215 ]
 [0.6300057 ]
 [0.61419785]
 [0.5878742 ]
 [0.60826737]
 [0.52088547]
 [0.4752857 ]
 [0.48963484]
 [0.44900575]
 [0.46227255]
 [0.44857684]
 [0.4634121 ]
 [0.4350326 ]
 [0.5122314 ]
 [0.52434754]
 [0.80704933]
 [0.5294758 ]
 [0.4903102 ]
 [0.41844267]
 [0.37936956]
 [0.3605583 ]
 [0.3975871 ]
 [0.40619248]
 [0.3

In [125]:
from sklearn.metrics import confusion_matrix
for i in range(4,7):
    threshold = i * 0.1
    binary_predictions = np.where(y_pred > threshold, 1, 0)
    cm = confusion_matrix(y_test, binary_predictions)
    print(cm)

[[13 83]
 [ 2 88]]
[[71 25]
 [48 42]]
[[87  9]
 [75 15]]


In [126]:
cm

array([[87,  9],
       [75, 15]], dtype=int64)